# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# Libraries
import numpy as np
import pandas as pd

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [2]:
df = pd.read_csv('Data/Admission_Predict.csv')
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


Remove trailing spaces at the end of the column names if there are any.


In [3]:
df.columns = df.columns.str.rstrip()
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


Let's evaluate the dataset by looking at the `head` function.

In [4]:
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [5]:
null_cols = df.isnull().sum()
null_cols
null_cols[null_cols > 0]

Series([], dtype: int64)

In [6]:
print(df.isnull().sum())
# print(df.isna().sum())

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64


In [7]:
df.isnull().values.any()

False

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [8]:
df.set_index('Serial No.')

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82
382,325,107,3,3.0,3.5,9.11,1,0.84
383,330,116,4,5.0,4.5,9.45,1,0.91


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [9]:
pd.Series([2, 7]).is_unique
#is this correct?

True

In [10]:
print(df.set_index(['GRE Score','CGPA']).index.is_unique)
#is this correct?

True


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [11]:
df.set_index(['GRE Score','CGPA'], inplace=True)

In [12]:
df.head()

,,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
GRE Score,CGPA,,,,,,,
337,9.65,1,118,4,4.5,4.5,1,0.92
316,8.00,2,104,3,3.0,3.5,1,0.72
322,8.67,3,110,3,3.5,2.5,1,0.80
314,8.21,4,103,2,2.0,3.0,0,0.65
330,9.34,5,115,5,4.5,3.0,1,0.90


In [13]:
df.reset_index(inplace=True)

In [14]:
df

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,1,118,4,4.5,4.5,1,0.92
1,316,8.00,2,104,3,3.0,3.5,1,0.72
2,322,8.67,3,110,3,3.5,2.5,1,0.80
3,314,8.21,4,103,2,2.0,3.0,0,0.65
4,330,9.34,5,115,5,4.5,3.0,1,0.90
...,...,...,...,...,...,...,...,...,...
380,324,9.04,381,110,3,3.5,3.5,1,0.82
381,325,9.11,382,107,3,3.0,3.5,1,0.84
382,330,9.45,383,116,4,5.0,4.5,1,0.91
383,312,8.78,384,103,3,3.5,4.0,0,0.67


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [15]:
rslt_df = df[(df['CGPA'] > 9) & df['Research'] == 1]

In [16]:
rslt_df

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,1,118,4,4.5,4.5,1,0.92
4,330,9.34,5,115,5,4.5,3.0,1,0.90
10,328,9.10,11,112,4,4.0,4.5,1,0.78
19,328,9.50,20,116,5,5.0,5.0,1,0.94
20,334,9.70,21,119,5,5.0,4.5,1,0.95
...,...,...,...,...,...,...,...,...,...
379,329,9.23,380,111,4,4.5,4.0,1,0.89
380,324,9.04,381,110,3,3.5,3.5,1,0.82
381,325,9.11,382,107,3,3.0,3.5,1,0.84
382,330,9.45,383,116,4,5.0,4.5,1,0.91


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [17]:
rslt_df2 = df[(df['CGPA'] > 9) & df['SOP'] < 3.5]

In [18]:
rslt_df2["Chance of Admit"].mean()

0.7239220779220773

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [19]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    
    # your code here
    the_mean = np.mean(col) 
    the_std = np.std(col)
    
    return (col - the_mean)/the_std

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [20]:
df['CGPA_std'] = standardize(df['CGPA'])

In [21]:
df['GRE_std'] = ((df['GRE Score'] - np.mean(df['GRE Score']))/np.std(df['GRE Score']))

In [22]:
df[['CGPA', 'GRE Score', 'LOR']]

,CGPA,GRE Score,LOR
0,9.65,337,4.5
1,8.00,316,3.5
2,8.67,322,2.5
3,8.21,314,3.0
4,9.34,330,3.0
...,...,...,...
380,9.04,324,3.5
381,9.11,325,3.5
382,9.45,330,4.5
383,8.78,312,4.0


In [23]:
df[['CGPA_std', 'GRE_std', 'LOR_std']] = df[['CGPA', 'GRE Score', 'LOR']].apply(standardize)

In [24]:
admission = df

In [25]:
admission.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338


We will generate the decision choice at random using the code below. Please run the cell.

In [26]:
# Libraries
from random import choices

In [27]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admission.shape[0])

#can you explain what exactly are we doing here, please? Are we just picking a random column from the last 3 colums that we created? 

In [28]:
admission.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338


Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [29]:
# your code here
admission['deciding_column'] = admission.lookup( admission.index,decision_choice)

In [30]:
admission

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.193197
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,0.076840
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,-1.039517
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.236698
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,-0.481338
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,324,9.04,381,110,3,3.5,3.5,1,0.82,0.736216,0.629546,0.076840,0.076840
381,325,9.11,382,107,3,3.0,3.5,1,0.84,0.852571,0.716170,0.076840,0.716170
382,330,9.45,383,116,4,5.0,4.5,1,0.91,1.417729,1.149292,1.193197,1.417729
383,312,8.78,384,103,3,3.5,4.0,0,0.67,0.304036,-0.409947,0.635019,0.635019


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [31]:
admission['decision'] = np.where(admission['deciding_column'] > 0.8, 1, 0)

In [32]:
admission

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.193197,1
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,0.076840,0
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,-1.039517,0
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.236698,0
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,-0.481338,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,324,9.04,381,110,3,3.5,3.5,1,0.82,0.736216,0.629546,0.076840,0.076840,0
381,325,9.11,382,107,3,3.0,3.5,1,0.84,0.852571,0.716170,0.076840,0.716170,0
382,330,9.45,383,116,4,5.0,4.5,1,0.91,1.417729,1.149292,1.193197,1.417729,1
383,312,8.78,384,103,3,3.5,4.0,0,0.67,0.304036,-0.409947,0.635019,0.635019,0


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [33]:
admission.loc[admission['decision'] == 1]

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.193197,1
10,328,9.10,11,112,4,4.0,4.5,1,0.78,0.835949,0.976043,1.193197,1.193197,1
19,328,9.50,20,116,5,5.0,5.0,1,0.94,1.500840,0.976043,1.751376,0.976043,1
20,334,9.70,21,119,5,5.0,4.5,1,0.95,1.833285,1.495789,1.193197,1.495789,1
21,336,9.80,22,119,5,4.0,3.5,1,0.97,1.999508,1.669038,0.076840,1.999508,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,340,9.74,370,113,4,5.0,5.0,1,0.96,1.899775,2.015536,1.751376,1.751376,1
370,335,9.82,371,117,5,5.0,5.0,1,0.96,2.032753,1.582414,1.751376,1.751376,1
377,326,9.12,378,112,4,4.0,3.5,1,0.84,0.869194,0.802794,0.076840,0.802794,1
379,329,9.23,380,111,4,4.5,4.0,1,0.89,1.052039,1.062667,0.635019,1.052039,1


In [48]:
accepted_applicants = admission[admission['decision'] == 1].copy()

In [49]:
accepted_applicants

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.193197,1
10,328,9.10,11,112,4,4.0,4.5,1,0.78,0.835949,0.976043,1.193197,1.193197,1
19,328,9.50,20,116,5,5.0,5.0,1,0.94,1.500840,0.976043,1.751376,0.976043,1
20,334,9.70,21,119,5,5.0,4.5,1,0.95,1.833285,1.495789,1.193197,1.495789,1
21,336,9.80,22,119,5,4.0,3.5,1,0.97,1.999508,1.669038,0.076840,1.999508,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,340,9.74,370,113,4,5.0,5.0,1,0.96,1.899775,2.015536,1.751376,1.751376,1
370,335,9.82,371,117,5,5.0,5.0,1,0.96,2.032753,1.582414,1.751376,1.751376,1
377,326,9.12,378,112,4,4.0,3.5,1,0.84,0.869194,0.802794,0.076840,0.802794,1
379,329,9.23,380,111,4,4.5,4.0,1,0.89,1.052039,1.062667,0.635019,1.052039,1


In [50]:
accepted_applicants["decision"][(accepted_applicants["decision"]==1)].count()

90

In [51]:
accepted_applicants.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.193197,1
10,328,9.10,11,112,4,4.0,4.5,1,0.78,0.835949,0.976043,1.193197,1.193197,1
19,328,9.50,20,116,5,5.0,5.0,1,0.94,1.500840,0.976043,1.751376,0.976043,1
20,334,9.70,21,119,5,5.0,4.5,1,0.95,1.833285,1.495789,1.193197,1.495789,1
21,336,9.80,22,119,5,4.0,3.5,1,0.97,1.999508,1.669038,0.076840,1.999508,1


# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [52]:
accepted_applicants.columns = accepted_applicants.columns.str.replace('.','').str.replace('\s','_').str.lower()

In [53]:
accepted_applicants.head()

,gre_score,cgpa,serial_no,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.193197,1
10,328,9.10,11,112,4,4.0,4.5,1,0.78,0.835949,0.976043,1.193197,1.193197,1
19,328,9.50,20,116,5,5.0,5.0,1,0.94,1.500840,0.976043,1.751376,0.976043,1
20,334,9.70,21,119,5,5.0,4.5,1,0.95,1.833285,1.495789,1.193197,1.495789,1
21,336,9.80,22,119,5,4.0,3.5,1,0.97,1.999508,1.669038,0.076840,1.999508,1


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [54]:
accepted_applicants['adjusted_gre'] = np.where(accepted_applicants['university_rating'] >= 4, accepted_applicants['gre_score'] + 10, accepted_applicants['gre_score'])

In [55]:
accepted_applicants.head(20)

,gre_score,cgpa,serial_no,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision,adjusted_gre
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.193197,1,347
10,328,9.10,11,112,4,4.0,4.5,1,0.78,0.835949,0.976043,1.193197,1.193197,1,338
19,328,9.50,20,116,5,5.0,5.0,1,0.94,1.500840,0.976043,1.751376,0.976043,1,338
20,334,9.70,21,119,5,5.0,4.5,1,0.95,1.833285,1.495789,1.193197,1.495789,1,344
21,336,9.80,22,119,5,4.0,3.5,1,0.97,1.999508,1.669038,0.076840,1.999508,1,346
28,338,9.40,29,118,4,3.0,4.5,1,0.91,1.334617,1.842287,1.193197,1.193197,1,348
29,340,9.60,30,114,5,4.0,4.0,1,0.90,1.667063,2.015536,0.635019,1.667063,1,350
30,331,9.80,31,112,5,4.0,5.0,1,0.94,1.999508,1.235916,1.751376,1.751376,1,341
31,320,9.20,32,110,5,5.0,5.0,1,0.88,1.002172,0.283048,1.751376,1.002172,1,330
39,332,9.10,40,117,4,4.5,4.0,0,0.87,0.835949,1.322541,0.635019,0.835949,1,342


In [56]:
accepted_applicants['adjusted_gre'] = pd.cut(accepted_applicants['gre_score'],4)
accepted_applicants

,gre_score,cgpa,serial_no,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision,adjusted_gre
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.193197,1,"(332.0, 340.0]"
10,328,9.10,11,112,4,4.0,4.5,1,0.78,0.835949,0.976043,1.193197,1.193197,1,"(324.0, 332.0]"
19,328,9.50,20,116,5,5.0,5.0,1,0.94,1.500840,0.976043,1.751376,0.976043,1,"(324.0, 332.0]"
20,334,9.70,21,119,5,5.0,4.5,1,0.95,1.833285,1.495789,1.193197,1.495789,1,"(332.0, 340.0]"
21,336,9.80,22,119,5,4.0,3.5,1,0.97,1.999508,1.669038,0.076840,1.999508,1,"(332.0, 340.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,340,9.74,370,113,4,5.0,5.0,1,0.96,1.899775,2.015536,1.751376,1.751376,1,"(332.0, 340.0]"
370,335,9.82,371,117,5,5.0,5.0,1,0.96,2.032753,1.582414,1.751376,1.751376,1,"(332.0, 340.0]"
377,326,9.12,378,112,4,4.0,3.5,1,0.84,0.869194,0.802794,0.076840,0.802794,1,"(324.0, 332.0]"
379,329,9.23,380,111,4,4.5,4.0,1,0.89,1.052039,1.062667,0.635019,1.052039,1,"(324.0, 332.0]"
